In [129]:
from string import Template
from functools import wraps

import numpy as np
import pandas as pd

from requests import Session

In [72]:
# .env variables / Config Files
mode = 'test' # 'test' or 'prod'

URL = {
    'test': 'https://sandbox.iexapis.com/stable',
    'prod': 'https://cloud.iexapis.com/stable',
}

PUBLIC_TOKEN = {
    'test': 'Tpk_014e57ad96e94b69b9de1f4a987829ff',
    'prod': 'pk_5e1051685cca40bdac58b98b417d526d',
}

SECRET_URL = {
    'test': 'Tsk_0c4b8f624f7e4ed2ac36ee422705797f',
    'prod': 'sk_c699b92eaee247b7803de8b79727adc3',
}

In [97]:
base_url = URL[mode]
token_params = {
    'token': PUBLIC_TOKEN[mode],
}

REQUEST_API = {
    'exchanges': '/ref-data/exchanges',
    'region_scripts': '/ref-data/region/$region/symbols',
    'exchange_scripts': '/ref-data/exchange/$exchange/symbols',
    'crypto_symbols': '/ref-data/crypto/symbols',
    'historical': '/stock/$symbol/chart/$range',
}

PARAM_DICT = {
    'get_historical_olhc_adjusted': {
        ''
    }
}

In [140]:
def get_data(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        url = func(*args, **kwargs)
        specific_params = PARAM_DICT.get(func.__name__,{})
        with Session() as s:
            r = s.get(url, params={**token_params, **specific_params})
            if r.status_code != 200:
                raise Exception(f'{r.status_code} error - Please check token / endpoints etc.')
            return pd.json_normalize(r.json())
    return wrapper

In [141]:
@get_data
def get_exchanges():
    return base_url + REQUEST_API['exchanges']

@get_data
def get_crypto_symbols():
    return base_url + REQUEST_API['crypto_symbols']

@get_data
def get_exchange_scripts(exchange_id='nse'):
    return base_url + Template(REQUEST_API['exchange_scripts']).substitute(exchange=exchange_id)

@get_data
def get_region_scripts(region_id='in'):
    return base_url + Template(REQUEST_API['region_scripts']).substitute(region=region_id)

@get_data
def get_historical_olhc_adjusted(symbol):
    return base_url + Template(REQUEST_API['historical']).substitute(symbol=symbol, range='max')



In [142]:
get_historical_olhc_adjusted('AAPL')


,close,high,low,open,symbol,volume,id,key,subkey,date,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
0,3.1300,3.1700,3.0720,3.0185,AAPL,1.088889e+09,AI_LTHIPRSEOCSRCI,LAAP,0,2006-12-29,...,86.11,40047606,2.6428,2.6139,2.7481,2.6661,1.101377e+09,"Dec 29, 06",0.000000,0.0000
1,3.0234,3.1622,3.0340,3.0900,AAPL,1.290408e+09,THSACEISPI_RCORLI,PAAL,0,2007-01-03,...,83.90,45670929,2.6242,2.6851,2.6817,2.5962,1.282084e+09,"Jan 3, 07",-0.038572,-0.0124
2,3.1429,3.0816,3.0326,3.0883,AAPL,8.714906e+08,ISORPCRHACES_TILI,LAPA,0,2007-01-04,...,87.95,31194034,2.6912,2.6417,2.7712,2.6690,8.799197e+08,"Jan 4, 07",0.067110,0.0228
3,3.1843,3.1696,3.0535,3.1980,AAPL,8.474784e+08,ICRTOPHSSICIL_EAR,ALAP,0,2007-01-05,...,88.20,31136941,2.6537,2.7196,2.6643,2.6703,8.734218e+08,"Jan 5, 07",-0.022336,-0.0074
4,3.0740,3.1321,3.1569,3.1500,AAPL,8.125617e+08,L_SIIROCEAHITSRPC,LAAP,0,2007-01-08,...,87.02,29454679,2.7149,2.7063,2.6990,2.7038,8.148339e+08,"Jan 8, 07",0.015531,0.0050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601,137.7700,137.0000,133.4750,135.6000,AAPL,9.101919e+07,TCALCORRSP_IISEHI,LPAA,0,2021-04-14,...,135.45,88974819,141.4300,134.2700,139.0000,134.3610,8.872299e+07,"Apr 14, 21",-2.481472,-0.0179
3602,136.0000,138.0000,135.3000,140.3800,AAPL,9.186903e+07,TIRCCOILISPERAS_H,LAAP,0,2021-04-15,...,133.88,90381354,139.1500,135.2000,139.0000,140.1000,9.297046e+07,"Apr 15, 21",2.583311,0.0190
3603,134.5200,140.8600,139.8700,139.4000,AAPL,8.679362e+07,HAPEOS_IRTIIRSCCL,ALAP,0,2021-04-16,...,137.89,86366041,140.9000,134.9700,141.1100,139.1900,8.741594e+07,"Apr 16, 21",-0.345237,-0.0025
3604,135.1800,140.3200,139.9400,137.1800,AAPL,9.788979e+07,IL_ECITASCHPIRSOR,LAPA,0,2021-04-19,...,134.95,96301319,133.5800,140.3100,141.0500,134.8600,9.448234e+07,"Apr 19, 21",0.699842,0.0053


In [144]:
a= {'a':1}
b=None

c={**a, **b}

TypeError: 'NoneType' object is not a mapping

In [126]:
df=get_exchanges()
df

('https://sandbox.iexapis.com/stable/ref-data/exchanges',
     exchange region                                        description   mic  \
 0       BSEX     AZ                                SnktEu gchoaBxeak c  XESB   
 1        DUB     IE                    lEekMAcai rnct toIrhSs ehxlka g  UDXB   
 2     HKHKSG     HK  c Ctnfn tgngo oLnnegKHHhkKcnonSO e hiE onekooi...  SCHS   
 3        CAR     VE                             ahScosaEkaxtc c aerCgn  AVCB   
 4        COL     LK                             o ohSomkoel CEaxgntcbc  LXOC   
 ..       ...    ...                                                ...   ...   
 175      PAR     FR                            oEtn eetrsxuaxE Prronui  PXRA   
 176    AUSSX     AU                      xcnEagS i  LiyteceeokdhtndSmy  PLAX   
 177     NEOE     CA                 r eaOMkoerNtEy ng   NceLdoe axBreh  ENOE   
 178      MUN     DE                                    ensuroMehB ecne  UXMN   
 179      ZIM     ZW                            abZ

In [122]:
GET = Template(REQUEST_API['historical']).substitute(region='in', exchange='nse', symbol='E1ABFRLPP-IS', range='max')
specific_params = {
    'chartCloseOnly':'true'
}

r = requests.get(BASE_URL + GET, params={**token_params, **specific_params})

In [104]:
r.url

'https://sandbox.iexapis.com/stable/stock/E1ABFRLPP-IS/chart/max?token=Tpk_014e57ad96e94b69b9de1f4a987829ff&chartCloseOnly=true'

In [105]:
BASE_URL + GET

'https://sandbox.iexapis.com/stable/stock/E1ABFRLPP-IS/chart/max'

In [106]:
exchanges = pd.json_normalize(r.json())

In [107]:
exchanges

,date,close,volume,change,changePercent,changeOverTime
0,2020-08-03,63.94,195240,0.000000,0.0000,0.000000
1,2020-08-04,64.41,147292,0.314886,0.0049,0.004913
2,2020-08-05,69.50,181318,3.533409,0.0555,0.062370
3,2020-08-06,69.02,40036,-0.732253,-0.0106,0.050015
4,2020-08-07,66.97,69186,-0.308066,-0.0047,0.045303
...,...,...,...,...,...,...
104,2021-01-01,106.80,53673,-0.780000,-0.0073,0.650429
105,2021-01-04,105.20,58411,0.104720,0.0010,0.661735
106,2021-01-05,103.17,97572,-0.780000,-0.0073,0.637286
107,2021-01-06,101.60,69429,-1.485295,-0.0146,0.598522
